In [52]:
#loading the dataset
import pandas as pd 
import numpy as np 
file="C:\\Users\\Hp\\Downloads\\Data Science\\entries_rr.xlsx"
data=pd.read_excel(file)

In [53]:
data.head()

,Unnamed: 0,KillZone,CODE,PT/CT,Long/Short,Choch H:TF,Conf:TF,SL Size,Entry in PD,SL in PD,Max RR,Day_of_week,outcome
0,942,LDN,NHI2,PT,L,15m,1mC,1.9,0.25<x<0.37,0<x<0.25,96.00,Wednesday,1
1,731,LDN,4HI2,PT,S,15m,1mC,2.7,0.37<x<0.5,0.25<x<0.37,60.59,Monday,1
2,987,NY,NHI2,PT,L,15m,1mC,5.0,0.62<x<0.75,0.37<x<0.5,57.38,Monday,1
3,0,LDN,1HOF4,PT,S,1H,1mC,2.4,0.5<x<0.62,0.25<x<0.37,53.58,Monday,1
4,435,LDN,NHI2,PT,S,15m,1mC,2.8,0.62<x<0.75,0.37<x<0.5,46.40,Monday,1


In [54]:
#lets check are there any missing values on this dataset
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 820 entries, 0 to 819
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   820 non-null    int64  
 1   KillZone     820 non-null    object 
 2   CODE         820 non-null    object 
 3   PT/CT        820 non-null    object 
 4   Long/Short   820 non-null    object 
 5   Choch H:TF   820 non-null    object 
 6   Conf:TF      820 non-null    object 
 7   SL Size      820 non-null    float64
 8   Entry in PD  820 non-null    object 
 9   SL in PD     820 non-null    object 
 10  Max RR       820 non-null    float64
 11  Day_of_week  820 non-null    object 
 12  outcome      820 non-null    int64  
dtypes: float64(2), int64(2), object(9)
memory usage: 83.4+ KB


In [55]:
#there are unwanted coloums as well.we remove them first.
data = data.drop(['Unnamed: 0', 'Long/Short', 'Choch H:TF','Conf:TF',"outcome",'Entry in PD','SL in PD'], axis=1)

In [56]:
data.head()

,KillZone,CODE,PT/CT,SL Size,Max RR,Day_of_week
0,LDN,NHI2,PT,1.9,96.00,Wednesday
1,LDN,4HI2,PT,2.7,60.59,Monday
2,NY,NHI2,PT,5.0,57.38,Monday
3,LDN,1HOF4,PT,2.4,53.58,Monday
4,LDN,NHI2,PT,2.8,46.40,Monday


In [57]:
sl_size=data[['SL Size']]
max_rr=data[['Max RR']]

In [58]:
#now our target variable is Max_RR
#in regression is good to do a feature scaling because it can help to improve model accurancy.
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
sl_size=scaler.fit_transform(sl_size)
max_rr=scaler.fit_transform(max_rr)


In [59]:
data["Max RR"]=max_rr

In [60]:
data['SL Size']=sl_size

In [61]:
data.head()

,KillZone,CODE,PT/CT,SL Size,Max RR,Day_of_week
0,LDN,NHI2,PT,0.012245,1.000000,Wednesday
1,LDN,4HI2,PT,0.028571,0.627263,Monday
2,NY,NHI2,PT,0.075510,0.593474,Monday
3,LDN,1HOF4,PT,0.022449,0.553474,Monday
4,LDN,NHI2,PT,0.030612,0.477895,Monday


In [62]:
#here we can see most of the datatype are categorical.
#first we need to convert these categorical into numerical
#so we use onehotencoding techniqques for this 

data=pd.get_dummies(data )
data = data.astype(int)
data["Max RR"]=max_rr
data['SL Size']=sl_size

In [63]:
#now we  need to group data into x and y.
x=data.drop("Max RR",axis=1)
y=data["Max RR"]

In [64]:
data.head()

,SL Size,Max RR,KillZone_LDN,KillZone_NY,CODE_1HOF13,CODE_1HOF2,CODE_1HOF4,CODE_1HOF5,CODE_4H4R2,CODE_4HI1,...,CODE_4HR2,CODE_NHI1,CODE_NHI2,PT/CT_CT,PT/CT_PT,Day_of_week_Friday,Day_of_week_Monday,Day_of_week_Thursday,Day_of_week_Tuesday,Day_of_week_Wednesday
0,0.012245,1.000000,1,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,1
1,0.028571,0.627263,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0
2,0.075510,0.593474,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,1,0,0,0
3,0.022449,0.553474,1,0,0,0,1,0,0,0,...,0,0,0,0,1,0,1,0,0,0
4,0.030612,0.477895,1,0,0,0,0,0,0,0,...,0,0,1,0,1,0,1,0,0,0


In [66]:
#now we need split our data inot train and test 
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25)

#now lets look how out data set were splitted 
x_train.shape,x_test.shape

((615, 21), (205, 21))

In [67]:
def model_acc(model):
    model.fit(x_train, y_train)
    acc = model.score(x_test, y_test)
    print(str(model)+ ' --> ' +str(acc))

In [68]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
model_acc(lr)

from sklearn.linear_model import Lasso
lasso = Lasso()
model_acc(lasso)

from sklearn.tree import DecisionTreeRegressor
dt = DecisionTreeRegressor()
model_acc(dt)

from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
model_acc(rf)

LinearRegression() --> 0.017091965402209253
Lasso() --> -0.0031904932248150164
DecisionTreeRegressor() --> -0.9350365124018944
RandomForestRegressor() --> 0.05283367610150391


In [71]:
from sklearn.linear_model import LinearRegression
import pandas as pd

# Assuming X and y are already defined
model = LinearRegression()
model.fit(x, y)

# Get predictions
y_pred = model.predict(x)

# Model evaluation
print("Intercept:", model.intercept_)
print("Coefficients:", model.coef_)
print("R² Score:", model.score(x, y))


Intercept: 0.07690234331743426
Coefficients: [-0.07427523  0.00339795 -0.00339795 -0.00959699 -0.00780873  0.0042532
 -0.01155776  0.00397151 -0.00856923  0.01302226 -0.02145103  0.00330262
 -0.00847432  0.04290848  0.00347294 -0.00347294 -0.01099935  0.01666205
  0.00175759 -0.00529155 -0.00212874]
R² Score: 0.07294506893761754


In [73]:
import statsmodels.api as sm

# Add a constant for the intercept
x = sm.add_constant(x)

# Fit the model
model = sm.OLS(y, x).fit()

# Summary of the model
print(model.summary())


                            OLS Regression Results                            
Dep. Variable:                 Max RR   R-squared:                       0.073
Model:                            OLS   Adj. R-squared:                  0.053
Method:                 Least Squares   F-statistic:                     3.712
Date:                Wed, 19 Feb 2025   Prob (F-statistic):           6.38e-07
Time:                        19:02:21   Log-Likelihood:                 890.36
No. Observations:                 820   AIC:                            -1745.
Df Residuals:                     802   BIC:                            -1660.
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                     0.03